# Teil 2 der Semesterarbeit: Parallel- und Zentralprojektionen

Deadline: Mittwoch,  21.10.2020

### 1. Parallelprojektion: https://www.youtube.com/watch?v=FXgTfIgZZZg

Achtung:  Parallelepiped (siehe wiki)

### 2. Zenralprojektion -> siehe Buch

## Theorie

Im Theorieteil müssen die mathematischen Grundlagen beschrieben werden, insbesondere die Idee und Anwendung der homogenen Koordinaten.


Aufgaben:
Sie verstehen die Matrizendarstellungen von Parallelprojektionen und Zentralprojektionen.
Sie können Zentralprojektionen mit numpy berechnen und mit matplotlib visualisieren.

Schreiben Sie eine Python Klasse für ein Parallelepiped (Spat).

Erstellen Sie eine Pythonfunktion, um ein Parallelepiped mit einer Paralleprojektion  auf der 𝑥𝑦
x
y
-Ebene darzustellen.
Die Funktion soll als Argumente ein Parallelepiped und einen Richtungsvektor verwenden; die Funktion sollte eine Matplotlib-Graphik erstellen.
Erstellen Sie eine Pythonfunktion, um ein Parallelepiped mit einer Zenralprojektion  auf der 𝑥𝑦
x
y
-Ebene darzustellen.
Die Funktion soll als Argumente ein Parallelepiped und ein Projektionszentrum (Kamera) verwenden; die Funktion sollte eine Matplotlib-Graphik erstellen.
Hilfsmittel: Numpy, Matplotlib, Jupyter Notebook (oder Python-IDE).

**Theorie:**

Mit der allgemeineren Form der homogenen Koordinaten, man kann damit eine weitere Klasse von nicht-linearen Abbildungen von $R2$ nach $R2$ als homogene Matrizen darstellen, und zwar **Zentralprojektionen**.

Beispiel:
imagexx1

Um einen Punkt $P$ zu projizieren, wird er mit dem Nullpunkt verbunden und
dann auf den Schnittpunkt $P′$ der Verbindungsgeraden mit der durch $y = d$
gegebenen Geraden abgebildet. Diese Abbildung ist nicht linear. Zum Bei-
spiel haben die beiden Punkte $(xP, yP + 1$) und $(xP, yP − 1)$ dieselbe Entfer-
nung von $P$. Aber ihre Projektionen werden nicht dieselbe Entfernung von $P′$
haben, d.h. Streckenverhältnisse auf Geraden bleiben nicht erhalten.

Obwohl die **Zentralprojektion nicht linear** ist, *lässt sie sich jedoch durch eine **homogene Matrix** darstellen*. 

Dazu muss man sich lediglich überlegen, wie die Koordinate $x′P$ des Punktes $P′$ berechnet wird.
Man hat es offenbar mit zwei ähnlichen rechtwinkligen Dreiecken zu tun, so dass die Verhältnisse $x′P/xP$ und $d/yP$ übereinstimmen. Das bedeutet, dass man $x′P$ durch $xP * (d/yP)$ berechnen kann. Jetzt multiplizieren wir $[xP, yP, 1]^T$ mit der folgenden Matrix:

M = $$\begin{pmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 1/d & 0 \end{pmatrix}$$

Das Ergebnis ist $[xP, yP, yP/d]^T$ 

Somit haben wir homogene Koordinaten für den Punkt $P′$ berechnet haben. Die homogene Matrix $M$ beschreibt also unsere **Zentralprojektion**.

In [2]:
################

In [3]:
# Erstellen Sie eine Pythonfunktion, um ein Parallelepiped mit einer Paralleprojektion  auf der 𝑥𝑦 Ebene darzustellen.

In [12]:
from scipy import *
from matrices import Matrix
from vectors import Vector
from canvas import Canvas
from homog import *
from math import pi
from time import sleep

In [13]:
left, right, bottom, top, near, far = -4, 4, -4, 4, 1, 20

In [14]:
# homogene Projektion, ich bekomme zurück eine homogene Matrix, die beschreibt wie meine Kamera sich bewegen wird

In [37]:
def projectionMatrix (xRotation, yRotation, distance):
    # bewege mich zuerst nach x, dann nach y und dann zurück Richtung 0,0 (distance)
    # dafür benötige ich eine Drehmatrix
    
    rotationMatrix = yRotationMatrix(yRotation) * xRotationMatrix(xRotation)
    
    # nun brauche ich die Richtung -> translation (um diesen Vektor von 0,0 weg zu nehmen)
    
    translation = rotationMatrix * (distance * Vector(0, 0, 1, 1))
    
    # vorher die Verschiebung dieser Vektor
    cameraMatrix = translationMatrix(-translation) * rotationMatrix 
    
    #ich muss von rechts nach links denken, erst skalieren, 20 / x Koordinaten, 20 / y Koordinaten und 20 / z Koord.
    ndcMatrix = scalingMatrix(20 / (right-left), 20 / (top-bottom), 20 / (far-near)) * translationMatrix ( -(left + right)/ 2, -(top +bottom) / 2, -(far + near) /2 )
    
    return ndcMatrix * cameraMatrix.invert()

In [16]:
# Auf eine Drehung, spielt der Ache der Drehung keine Rolle (0), d.h. nur die andere zwei Achsen
# yRotation -> gibt zurück eine Drehungsmatrix, wo der Achse der Drehung keine Rolle spielt (0)
yRotationMatrix(pi/6)

M[[0.8660254037844387, 0, 0.49999999999999994, 0], [0, 1, 0, 0], [-0.49999999999999994, 0, 0.8660254037844387, 0], [0, 0, 0, 1]]

In [27]:
# für einen Würfel, ich nehme die Koordinaten für einen Würfel (mit -2, 2 x und y Koordinaten)
cubePoints = [Vector(x, y, z) for x in [-2, 2] for y in [-2, 2] for z in [-2, 2]]

In [28]:
cubePoints

[v(-2, -2, -2),
 v(-2, -2, 2),
 v(-2, 2, -2),
 v(-2, 2, 2),
 v(2, -2, -2),
 v(2, -2, 2),
 v(2, 2, -2),
 v(2, 2, 2)]

In [29]:
# um die Kanten auch zu zeichnen, eine Funktion die auf alle Paare geht und sagt wenn eure Abstand 4 ist seid ihr eine Kante

cubeEdges = []
for i,  P1 in enumerate(cubePoints):
    for P2 in cubePoints[i+1:]:
        if (P1 - P2).norm() == 4:
            #dort wo der Abstand gleich 4 ist
            cubeEdges.append([P1, P2])

In [30]:
cubeEdges  # das ist die Liste aller Kanten des Würfels

[[v(-2, -2, -2), v(-2, -2, 2)],
 [v(-2, -2, -2), v(-2, 2, -2)],
 [v(-2, -2, -2), v(2, -2, -2)],
 [v(-2, -2, 2), v(-2, 2, 2)],
 [v(-2, -2, 2), v(2, -2, 2)],
 [v(-2, 2, -2), v(-2, 2, 2)],
 [v(-2, 2, -2), v(2, 2, -2)],
 [v(-2, 2, 2), v(2, 2, 2)],
 [v(2, -2, -2), v(2, -2, 2)],
 [v(2, -2, -2), v(2, 2, -2)],
 [v(2, -2, 2), v(2, 2, 2)],
 [v(2, 2, -2), v(2, 2, 2)]]

In [34]:
def drawCube(c, xAngleDeg = 20, yAngleDeg = 30, distance = 10, edges = cubeEdges):

        M = projectionMatrix(xAngleDeg / 180 * pi, yAngleDeg / 180 * pi, distance)
    
        c.clear()
        for P1, P2 in edges:
            P1= lower(M * lift(P1))
            P2 = lower(M * lift(P2))
            c.drawSegment(P1, P2)
            
c = Canvas()
            
            

<IPython.core.display.Javascript object>

In [38]:
drawCube(c)

<IPython.core.display.Javascript object>

In [40]:
for angle in range(-20, 20):
    drawCube(c, yAngleDeg=angle)
    sleep(0.1)

<IPython.core.display.Javascript object>